In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

import nltk
import string
import pandas as pd
import numpy as np

#download assets from nltk
#nltk.download('stopwords')
#nltk.download('punkt')

def tfidf(corpus):
    '''
    Computes the TF-IDF (term frequency - inverse document frequency) matrix

    Args
    - corpus: a list of documents

    Returns
    - tfidfVec: an m x n matrix of the corpus. m = number of different terms used in the documents, n = number of documents 
    - vocab: all the unique words used in the corpus, excluding stop words
    '''

    #vectorizer = TfidfVectorizer(stop_words = stopwords.words('english'))
    vectorizer = CountVectorizer(stop_words='english')
    tfidfVec = vectorizer.fit_transform(corpus)
    vocab = vectorizer.get_feature_names()
        
    return tfidfVec, vocab

def svd(tfidfVec):
    '''
    Gives the singular value decomposition of an m x n matrix.
    A = U * sigma * V^t
    
    Args
    - tfidfVec: an m x n matrix. m = number of documents or sentences, n = number of terms

    Returns
    - U: an m x r matrix of left singular values (document-topic table). r = number of topics
    - sigma: an r x r diagonal matrix of singular values in decreasing order across the diagonal
    - V^t: an n x r matrix of right singular values (term-topic table)
    '''

    lsa = TruncatedSVD(n_components = 10, n_iter=20)
    u = lsa.fit_transform(tfidfVec)
    sigma = lsa.singular_values_
    vt = lsa.components_.T

    return u, sigma, vt

def getImportantSentences(u, sigma):
    '''
    Uses the LSA enhancement described by Josef Steinberg, et al.
    Take all topics that have singular values > half of the largest singular value

    Compute sk = sqrt(sum(v_ki^2 * sigma_i^2) from i = 1 to n)
    sk is the length of the vector of the kth sentence
    n is the number of topics 
    '''
    #look for the sigma value range that we need to consider using binary search
    #sigma array is sorted in descending order and will never be empty
    l, r, target = 0, len(sigma), sigma[0]/2
    while l < r:
        mid = l + (r-l)//2

        if sigma[mid] < target:
            r = mid
        else:
            l = mid + 1
    sigmaBound = l

    uSlice = u[:, :sigmaBound]
    sigmaSlice = sigma[:sigmaBound]
    uSq = np.square(uSlice)
    sigSq = np.square(np.diag(sigmaSlice))
    prod = np.matmul(uSq, sigSq)
    result = np.sqrt(np.sum(prod, axis = 1)).T

    return (-result).argsort()

def createWordToSentenceMap(corpus):
    wordToSentence = {}
    stopWords = set(stopwords.words('english'))

    for i, doc in enumerate(corpus):
        #remove punctuation while preserving contractions in text
        sanitizeText = doc.translate(str.maketrans('', '', string.punctuation))
        tokenized = word_tokenize(sanitizeText)
        #remove duplicate words
        tokenized = list(set([word.lower() for word in tokenized]))

        for word in tokenized:
            if word not in stopWords:
                if word not in wordToSentence:
                    wordToSentence[word] = [i]
                else:
                    wordToSentence[word].append(i)
    
    return wordToSentence

def extractSummary(u, sigma, k, corpus):
    '''
    Summary will be taken from the top k sentences from getImportantSentences()
    for each topic.
    '''
    return [corpus[i] for i in getImportantSentences(u, sigma)[:k]]


In [30]:
def preProcess(blockText):
    tokenized = sent_tokenize(blockText) 
    return [token.replace('\n',' ') for token in tokenized]

In [31]:
text = u'''
The list of businesses impacted by a lockdown beginning Monday in Toronto and Peel Region were not clearly communicated, the owner of a Toronto massage spa says.

While the Ontario government offered a partial list of what would remain open after the COVID-19 shutdown begins at 12:01 a.m., Kate Armstrong, owner and director of Bahn Thai Spa, told the Star she was unsure whether her business would be impacted.

The Ontario government’s late-afternoon announcement on Friday stated that personal services, such as nail and hair salons, would now be closed. Missing, however, were details of all services included in the shutdown.

However, on Sunday, the Ministry of Health confirmed to the Star that “regulated health professionals including dentists, optometrists, chiropractic services, ophthalmologists, physical and occupational therapists and podiatrists will be able to operate.”

A spokesperson said that “under lockdown, regulated health professionals, including massage therapists, will be able to operate. Regulated health professionals such as registered massage therapists were not impacted and therefore not referenced.”

Working “in partnership with the chief medical officer of health and our local medical officers of health, we continue to closely monitor the evolving situation to advise if and when public health measures need to be adjusted,” the spokesperson also said.

In Ontario’s first lockdown last spring, physiotherapy, chiropractic services and massage therapists were among those to close their doors, which left some confused about what is happening this time around.

“We have to continue to communicate with clients that are calling and saying, ‘Are we seeing you on Monday or not?’ We’re having to say we’ll call you as soon as we know something more,” Armstrong said.

“It’s not like a haircut,” she said, adding that people are often seeking massage to treat physical pain or for mental health care.

To Armstrong, massage has been as important as mental healthcare for Ontarians during the months-long pandemic. “I see the fatigue setting in on everyone’s faces ... The stress is so high … right now, (with) people not being able to be with their families. It’s so important to have human touch.”

The Ontario Physiotherapy Association shared the news that physiotherapy services would be able to continue operations, said Shafiq Bhanji, president of Athlete’s Care Sports Medicine Centres.

“We received direction from our respective colleges and professional associations on Friday and over the weekend via email indicating that our services would not be impacted the upcoming lockdown,” Bhanji said in an email to the Star.

While Bhanji was able to confirm that Athlete’s Care could continue offering services and communicate that to clients via email, patients are still reaching out to verify whether they can keep their appointments.

“It seems there was a fair bit of confusion in the general public about whether or not these services would be impacted,” Bhanji said. “... We are fortunate that our colleges and professional associations acted quickly to inform their members.”
'''

corpus = preProcess(text)
display(corpus)
tfidfVec, vocab = tfidf(corpus)
wordToSentence = createWordToSentenceMap(corpus)
print(vocab)
print(wordToSentence)
print(tfidfVec)
print('----------------------------------------------------------')

u, sigma, vt = svd(tfidfVec)
numTopics = u.shape[1] + 1

dfSVD = pd.DataFrame(u, columns=[f'topic{str(i)}' for i in range(1, numTopics)])
docCol = pd.DataFrame({'Documents': corpus})
dfSVD = pd.concat([docCol, dfSVD], axis = 1)

display(dfSVD)
print('----------------------------------------------------------')
print(sigma)

print('----------------------------------------------------------')

dfVt = pd.DataFrame(vt, columns=[f'topic{str(i)}' for i in range(1, numTopics)])
vocabCol = pd.DataFrame({'Terms': vocab})
dfVt = pd.concat([vocabCol, dfVt], axis = 1)

display(dfVt)

for i in range(1, numTopics):
    dfVtSort = dfVt.sort_values(by=f'topic{i}', ascending=False)
    display(dfVtSort[['Terms', f'topic{i}']])
print('----------------------------------------------------------')


#df = pd.DataFrame()
#print(df)

[' The list of businesses impacted by a lockdown beginning Monday in Toronto and Peel Region were not clearly communicated, the owner of a Toronto massage spa says.',
 'While the Ontario government offered a partial list of what would remain open after the COVID-19 shutdown begins at 12:01 a.m., Kate Armstrong, owner and director of Bahn Thai Spa, told the Star she was unsure whether her business would be impacted.',
 'The Ontario government’s late-afternoon announcement on Friday stated that personal services, such as nail and hair salons, would now be closed.',
 'Missing, however, were details of all services included in the shutdown.',
 'However, on Sunday, the Ministry of Health confirmed to the Star that “regulated health professionals including dentists, optometrists, chiropractic services, ophthalmologists, physical and occupational therapists and podiatrists will be able to operate.”  A spokesperson said that “under lockdown, regulated health professionals, including massage th

['01', '12', '19', 'able', 'acted', 'adding', 'adjusted', 'advise', 'afternoon', 'announcement', 'appointments', 'armstrong', 'association', 'associations', 'athlete', 'bahn', 'beginning', 'begins', 'bhanji', 'bit', 'business', 'businesses', 'calling', 'care', 'centres', 'chief', 'chiropractic', 'clearly', 'clients', 'close', 'closed', 'closely', 'colleges', 'communicate', 'communicated', 'confirm', 'confirmed', 'confused', 'confusion', 'continue', 'covid', 'dentists', 'details', 'direction', 'director', 'doors', 'email', 'evolving', 'faces', 'fair', 'families', 'fatigue', 'fortunate', 'friday', 'general', 'government', 'hair', 'haircut', 'happening', 'having', 'health', 'healthcare', 'high', 'human', 'impacted', 'important', 'included', 'including', 'indicating', 'inform', 'kate', 'know', 'late', 'left', 'like', 'list', 'll', 'local', 'lockdown', 'long', 'massage', 'measures', 'medical', 'medicine', 'members', 'mental', 'ministry', 'missing', 'monday', 'monitor', 'months', 'nail', 'ne

,Documents,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10
0,The list of businesses impacted by a lockdown...,5.733549e-01,8.396878e-01,1.675249e+00,9.779961e-01,6.480998e-01,3.351836e+00,-2.314502e-01,-1.803172e-01,-8.540333e-01,-7.734972e-01
1,While the Ontario government offered a partial...,4.181120e-01,1.884095e+00,3.571864e+00,2.109817e+00,-1.259232e+00,-1.414688e+00,4.204671e-01,-4.004819e-01,9.520558e-02,1.491290e-01
2,The Ontario government’s late-afternoon announ...,2.451232e-01,1.041237e+00,1.966595e-01,7.512446e-02,4.942955e-02,-7.973867e-01,-1.289564e+00,2.378686e+00,3.045175e-01,-1.614288e+00
3,"Missing, however, were details of all services...",1.654312e-01,4.533973e-01,1.429418e-02,4.687537e-02,4.012813e-02,-2.055919e-01,-1.677731e-01,2.454340e-01,-4.758368e-02,-9.171092e-02
4,"However, on Sunday, the Ministry of Health con...",6.068741e+00,1.083800e-01,-2.187905e+00,2.620222e+00,-1.279322e-01,-2.063247e-01,4.429118e-01,-1.341970e-01,-2.737936e-01,-5.990454e-02
5,Regulated health professionals such as registe...,5.373233e+00,-2.193546e+00,2.210515e+00,-2.441951e+00,6.425418e-03,-2.558828e-01,-4.330012e-01,1.085281e-01,-3.447424e-01,6.541710e-02
6,"In Ontario’s first lockdown last spring, physi...",1.005183e+00,1.226459e+00,-3.430399e-01,3.456052e-01,-3.179000e-01,8.269286e-01,-1.777434e+00,1.303354e+00,3.604810e-01,1.688418e+00
7,“We have to continue to communicate with clien...,4.329970e-01,7.554828e-01,3.575215e-01,-7.246776e-01,-1.043790e-01,5.975294e-01,2.552526e+00,1.660226e+00,1.657622e-01,1.295077e+00
8,"“It’s not like a haircut,” she said, adding th...",1.280971e+00,1.328839e-01,1.316760e-02,-3.708873e-01,-1.904938e-01,7.271533e-01,4.299136e-01,-4.808293e-01,2.842603e+00,-9.264928e-01
9,"To Armstrong, massage has been as important as...",2.770382e-01,2.568594e-01,2.427280e-01,3.078552e-03,-3.491427e-01,5.982232e-01,1.811334e-01,8.735893e-02,1.398263e+00,3.558566e-01


----------------------------------------------------------
[8.50185931 5.73182996 5.20315806 5.02358551 4.39296778 4.07909479
 3.90983219 3.54351125 3.49529473 3.29637809]
----------------------------------------------------------


,Terms,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10
0,01,0.005784,0.057348,0.131935,0.083602,-0.065251,-0.085022,0.027505,-0.031894,0.007793,0.013724
1,12,0.005784,0.057348,0.131935,0.083602,-0.065251,-0.085022,0.027505,-0.031894,0.007793,0.013724
2,19,0.005784,0.057348,0.131935,0.083602,-0.065251,-0.085022,0.027505,-0.031894,0.007793,0.013724
3,able,0.195827,0.169287,-0.228551,0.082371,-0.114621,-0.021380,0.130064,-0.090008,-0.124838,-0.146238
4,acted,0.000619,0.008378,0.001913,-0.002492,0.046556,-0.015214,-0.007887,-0.031172,0.019633,0.045290
...,...,...,...,...,...,...,...,...,...,...,...
161,unsure,0.005784,0.057348,0.131935,0.083602,-0.065251,-0.085022,0.027505,-0.031894,0.007793,0.013724
162,upcoming,0.013272,0.069406,0.012165,-0.014315,0.175333,-0.043812,-0.019156,-0.047345,0.027599,0.043269
163,verify,0.009545,0.057824,-0.022467,-0.045602,0.014070,-0.011487,0.102210,0.034737,-0.067476,-0.109774
164,weekend,0.013272,0.069406,0.012165,-0.014315,0.175333,-0.043812,-0.019156,-0.047345,0.027599,0.043269


...

,Terms,topic7
33,communicate,0.269186
28,clients,0.269186
11,armstrong,0.206330
39,continue,0.195390
22,calling,0.166976
...,...,...
156,time,-0.116273
78,lockdown,-0.121596
139,services,-0.151378
116,physiotherapy,-0.207214


,Terms,topic8
139,services,0.199143
8,afternoon,0.189439
9,announcement,0.189439
151,stated,0.189439
133,salons,0.189439
...,...,...
64,impacted,-0.088912
150,star,-0.089927
23,care,-0.089948
3,able,-0.090008


,Terms,topic9
85,mental,0.347126
80,massage,0.256099
113,people,0.247616
5,adding,0.232674
107,pain,0.232674
...,...,...
128,regulated,-0.073039
14,athlete,-0.094958
39,continue,-0.109608
3,able,-0.124838


,Terms,topic10
116,physiotherapy,0.183437
11,armstrong,0.165658
58,happening,0.155384
73,left,0.155384
37,confused,0.155384
...,...,...
30,closed,-0.148562
133,salons,-0.148562
72,late,-0.148562
9,announcement,-0.148562


----------------------------------------------------------


In [33]:
sentences = extractSummary(u, sigma, 5, corpus)
summary = '\n\n'.join(sentences)
print(summary)


However, on Sunday, the Ministry of Health confirmed to the Star that “regulated health professionals including dentists, optometrists, chiropractic services, ophthalmologists, physical and occupational therapists and podiatrists will be able to operate.”  A spokesperson said that “under lockdown, regulated health professionals, including massage therapists, will be able to operate.

Regulated health professionals such as registered massage therapists were not impacted and therefore not referenced.”  Working “in partnership with the chief medical officer of health and our local medical officers of health, we continue to closely monitor the evolving situation to advise if and when public health measures need to be adjusted,” the spokesperson also said.

It’s so important to have human touch.”  The Ontario Physiotherapy Association shared the news that physiotherapy services would be able to continue operations, said Shafiq Bhanji, president of Athlete’s Care Sports Medicine Centres.

Wh